## Seasons

In [1]:
from skyfield import api
ts = api.load.timescale()
eph = api.load('de421.bsp')
from skyfield import almanac
from skyfield.framelib import itrs

In [2]:
t0 = ts.utc(2021, 1, 1)
t1 = ts.utc(2021, 12, 31)
t, y = almanac.find_discrete(t0, t1, almanac.seasons(eph))

In [3]:
for yi, ti in zip(y, t):
    print(yi, almanac.SEASON_EVENTS[yi], ti.utc_iso(' '))

0 Vernal Equinox 2021-03-20 09:37:29Z
1 Summer Solstice 2021-06-21 03:32:10Z
2 Autumnal Equinox 2021-09-22 19:21:06Z
3 Winter Solstice 2021-12-21 15:59:18Z


## Phases of moon

In [4]:
t0 = ts.utc(2021, 1, 1)
t1 = ts.utc(2021, 1, 31)
t, y = almanac.find_discrete(t0, t1, almanac.moon_phases(eph))

In [5]:
print(t.utc_iso())
print(y)
print([almanac.MOON_PHASES[yi] for yi in y])

['2021-01-06T09:37:13Z', '2021-01-13T05:00:11Z', '2021-01-20T21:01:37Z', '2021-01-28T19:16:14Z']
[3 0 1 2]
['Last Quarter', 'New Moon', 'First Quarter', 'Full Moon']


## Lunar nodes

In [6]:

t2, y2 = almanac.find_discrete(t0, t1, almanac.moon_nodes(eph))

print(t2.utc_iso())
print(y2)
print([almanac.MOON_NODES[yi] for yi in y2])

['2021-01-10T20:15:17Z', '2021-01-24T21:47:04Z']
[0 1]
['descending', 'ascending']


## Opposition and Conjuction

In [7]:
t00 = ts.utc(2020, 1, 1)
t11 = ts.utc(2024, 1, 1)
f = almanac.oppositions_conjunctions(eph, eph['mars'])
t3, y3 = almanac.find_discrete(t00, t11, f)

print(t3.utc_iso())
print(y3)

['2020-10-13T23:25:47Z', '2021-10-08T04:00:52Z', '2022-12-08T05:41:37Z', '2023-11-18T05:41:37Z']
[1 0 1 0]


#### any planets outside earth's orbit; 0 means moment of conjunction and 1 means moment of opposition 
#### and its opposite for the moon

## meridian transits

In [8]:
bluffton = api.wgs84.latlon(+27.3314, +88.6138)

t0 = ts.utc(2021, 1, 22)
t1 = ts.utc(2021, 1, 23)


In [9]:
f = almanac.meridian_transits(eph, eph['Mars'], bluffton)
t, y = almanac.find_discrete(t0, t1, f)

In [10]:
print(t.utc_strftime('%Y-%m-%d %H:%M'))
print(y)
print([almanac.MERIDIAN_TRANSITS[yi] for yi in y])

['2021-01-22 00:18', '2021-01-22 12:17']
[0 1]
['Antimeridian transit', 'Meridian transit']


## Sunrise and Sunset

In [11]:
t0 = ts.utc(2021, 1, 23, 4)
t1 = ts.utc(2021, 1, 24, 4)
t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(eph, bluffton))

print(t.utc_iso())
print(y)

['2021-01-23T11:39:42Z', '2021-01-24T00:55:01Z']
[0 1]


## Sun up in polar regions

In [12]:
far_north = api.wgs84.latlon(89, -80) #polar regions
f = almanac.sunrise_sunset(eph, far_north)
t0 = ts.utc(2021, 1, 1, 4)
t1 = ts.utc(2021, 12, 31, 4)
t, y = almanac.find_discrete(t0, t1, f)

In [13]:
tr=ts.utc(2021, range(1,13), 1, 4)
print(t.utc_iso())  # Empty list: no sunrise or sunset
print(f(tr))        # But we can ask if the sun is up

for i in range(len(tr)):
    if f(tr[i]):
        print('polar day')
    else:
        print('polar night')

['2021-03-16T14:18:01Z', '2021-03-16T21:12:12Z', '2021-03-17T12:39:25Z', '2021-03-17T22:56:56Z', '2021-03-18T11:12:48Z', '2021-03-19T00:34:26Z', '2021-03-19T09:43:59Z', '2021-03-20T02:29:37Z', '2021-03-20T07:53:58Z', '2021-09-23T02:16:11Z', '2021-09-23T08:40:35Z', '2021-09-24T00:35:41Z', '2021-09-24T10:26:31Z', '2021-09-24T23:09:38Z', '2021-09-25T12:01:58Z', '2021-09-25T21:43:17Z', '2021-09-26T13:49:19Z', '2021-09-26T20:01:13Z']
[False False False  True  True  True  True  True  True False False False]
polar night
polar night
polar night
polar day
polar day
polar day
polar day
polar day
polar day
polar night
polar night
polar night


## Lunar eclipse

In [14]:
from skyfield import eclipselib

In [15]:
t0 = ts.utc(2021, 1, 1)
t1 = ts.utc(2022, 1, 1)
t, y, details = eclipselib.lunar_eclipses(t0, t1, eph)

In [16]:
for ti, yi in zip(t, y):
    print(ti.utc_strftime('%Y-%m-%d %H:%M'),
          'y={}'.format(yi),
          eclipselib.LUNAR_ECLIPSES[yi])

2021-05-26 11:19 y=2 Total
2021-11-19 09:03 y=1 Partial


## Solar Eclipse

In [17]:
t = ts.utc(2021, 1,1,0,range(365*1440))
eph = api.load('de421.bsp')
sun, moon, earth = eph['sun'], eph['moon'], eph['earth']
e = earth.at(t)
s = e.observe(sun).apparent()
m = e.observe(moon).apparent()
saps=s.separation_from(m)

In [18]:
for i,ttr in zip(saps.degrees,t):
     if i>=0 and i<=0.85:
            print(i,ttr.utc_jpl())
    

0.8485746965710017 A.D. 2021-Jun-10 10:16:00.0000 UTC
0.8468223554892818 A.D. 2021-Jun-10 10:17:00.0000 UTC
0.8451354974086399 A.D. 2021-Jun-10 10:18:00.0000 UTC
0.8435145164222478 A.D. 2021-Jun-10 10:19:00.0000 UTC
0.8419597942542035 A.D. 2021-Jun-10 10:20:00.0000 UTC
0.84047169983082 A.D. 2021-Jun-10 10:21:00.0000 UTC
0.8390505889095518 A.D. 2021-Jun-10 10:22:00.0000 UTC
0.8376968036237101 A.D. 2021-Jun-10 10:23:00.0000 UTC
0.8364106721342457 A.D. 2021-Jun-10 10:24:00.0000 UTC
0.8351925082361025 A.D. 2021-Jun-10 10:25:00.0000 UTC
0.8340426109803337 A.D. 2021-Jun-10 10:26:00.0000 UTC
0.8329612643435625 A.D. 2021-Jun-10 10:27:00.0000 UTC
0.8319487368843327 A.D. 2021-Jun-10 10:28:00.0000 UTC
0.8310052814118298 A.D. 2021-Jun-10 10:29:00.0000 UTC
0.8301311346779157 A.D. 2021-Jun-10 10:30:00.0000 UTC
0.8293265171087493 A.D. 2021-Jun-10 10:31:00.0000 UTC
0.8285916325182882 A.D. 2021-Jun-10 10:32:00.0000 UTC
0.8279266678470177 A.D. 2021-Jun-10 10:33:00.0000 UTC
0.8273317929552245 A.D. 2021-J

In [19]:
#0.8353711228229 A.D. 2021-Jun-10 10:59:00.0000 UTC

#0.9733651648249619 A.D. 2021-Dec-04 07:34:00.0000 UTC
        

In [3]:

t = ts.utc(2021,1,10,10,59)
sun, earth =eph['sun'], eph['earth']
p = earth.at(t).observe(sun).apparent()
lat, lon, distance = p.frame_latlon(itrs)
lon_degrees = (lon.degrees + 180.0) % 360.0 - 180.0
print('Sub-solar latitude: {:.1f} degrees'.format(lat.degrees))
print('Sub-solar longitude: {:.1f} degrees'.format(lon_degrees))

Sub-solar latitude: -21.9 degrees
Sub-solar longitude: 17.1 degrees


In [4]:

t = ts.utc(2021,12,4,7,34)
sun, earth =eph['sun'], eph['earth']
p = earth.at(t).observe(sun).apparent()
lat, lon, distance = p.frame_latlon(itrs)
lon_degrees = (lon.degrees + 180.0) % 360.0 - 180.0
print('Sub-solar latitude: {:.1f} degrees'.format(lat.degrees))
print('Sub-solar longitude: {:.1f} degrees'.format(lon_degrees))

Sub-solar latitude: -22.3 degrees
Sub-solar longitude: 64.0 degrees
